In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pyet
import numpy as np
from scipy import stats

In [ ]:
#read data
data_16412 = pd.read_csv('klima_daily.csv', index_col=1, parse_dates=True)
data_16412

In [ ]:
# Convert Glabalstrahlung J/cm2 to MJ/m2 by dividing to 100

meteo = pd.DataFrame({"time":data_16412.index, "tmean":data_16412.t, "tmax":data_16412.tmax, "tmin":data_16412.tmin, "rh":data_16412.rel, 
                      "wind":data_16412.vv, "rs":data_16412.strahl/100})
time, tmean, tmax, tmin, rh, wind, rs = [meteo[col] for col in meteo.columns]

lat = 47.077778*np.pi/180  # Latitude of the meteorological station, converting from degrees to radians
elevation = 367  # meters above sea-level

lambda1=pyet.calc_lambda(tmean=tmean)  # Latent Heat of Vaporization in PyEt [MJ kg-1] 
lambda0 = 2.45  # Latent Heat of Vaporization in Guo et al., 2016 [MJ kg-1] 
lambda_corr = lambda1 / lambda0  # Correction factor

In [ ]:
import numpy as np

def sensitivity_analysis(func, param_name, param_range, fixed_params, num_samples=200, apply_correction = False):
    """
    Conducts a sensitivity analysis on a given function.

    :param func: The function to test.
    :param param_name: The parameter name as a string that we want to analyze.
    :param param_range: A tuple (min, max) specifying the range of values.
    :param fixed_params: A dictionary of other parameters that will be kept constant.
    :return: A list of output values for each sampled value in param_range.
    """
    sampled_values = np.linspace(param_range[0], param_range[1], num_samples)
    results = []

    lambda0 = 2.45  # Latent Heat of Vaporization in Guo et al., 2016 [MJ kg-1] 

    for value in sampled_values:
        # Update the parameter of interest.
        params = fixed_params.copy()
        params[param_name] = value

        # Get the result of the function with the current parameter set.

        if apply_correction is False:
            labmda_corr = 1
        else:
            lambda1 = pyet.calc_lambda(params['tmean'])
            lambda_corr = lambda1/lambda0

        result = func(**params) * lambda_corr
        results.append(result)

    return results

# Example usage:
fixed_parameters = {
    'tmean': tmean_value,
    'wind': wind_value,
    # ... other fixed parameters ...
    'albedo': 0.08
}

results = sensitivity_analysis(pyet.penman, 'wind', (0.5, 10.5), fixed_parameters)
